In [1]:
import torch
from torchtext.data.utils  import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import io
from typing import Iterable, List

In [2]:
!pip list|grep torch

torch                    2.0.0
torch-tb-profiler        0.4.0
torchaudio               0.13.0
torchdata                0.6.0
torchinfo                1.7.1
torchmetrics             0.11.4
torchnet                 0.0.4
torchtext                0.15.1
torchvision              0.15.1


In [3]:
# from torchtext.datasets import multi30k, Multi30k

# # We need to modify the URLs for the dataset since the links to the original dataset are broken
# # Refer to https://github.com/pytorch/text/issues/1756#issuecomment-1163664163 for more info
# multi30k.URL["train"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/training.tar.gz"
# multi30k.URL["valid"] = "https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/validation.tar.gz"

# SRC_LANGUAGE = 'de'
# TGT_LANGUAGE = 'en'

# # Place-holders
# token_transform = {}
# vocab_transform = {}

# token_transform[SRC_LANGUAGE] = get_tokenizer('spacy', language='de_core_news_md')
# token_transform[TGT_LANGUAGE] = get_tokenizer('spacy', language='en_core_web_md')


# # helper function to yield list of tokens
# def yield_tokens(data_iter: Iterable, language: str) -> List[str]:
#     language_index = {SRC_LANGUAGE: 0, TGT_LANGUAGE: 1}

#     for data_sample in data_iter:
#         yield token_transform[language](data_sample[language_index[language]])

# # Define special symbols and indices
# UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
# # Make sure the tokens are in order of their indices to properly insert them in vocab
# special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

# for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
#     # Training data Iterator
#     train_iter = Multi30k(root='/home/uih/JYL/GitHub/Transformer/data', split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
#     # Create torchtext's Vocab object
#     vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
#                                                     min_freq=1,
#                                                     specials=special_symbols,
#                                                     special_first=True)

# # Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# # If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
# for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
#   vocab_transform[ln].set_default_index(UNK_IDX)

In [4]:
# for i in train_iter:
#     print(i)
#     break

In [5]:
SRC_LANGUAGE = 'zh'
TGT_LANGUAGE = 'en'

token_transform = {}
vocab_transform = {}

In [6]:
token_transform[SRC_LANGUAGE]= get_tokenizer('spacy', language='zh_core_web_md')
token_transform[TGT_LANGUAGE]= get_tokenizer('spacy', language='en_core_web_md')

In [7]:
def yield_tokens(data_iter:Iterable, language:str) -> List[str]:
    language_index = {SRC_LANGUAGE:0, TGT_LANGUAGE: 1}
    for data_sample in data_iter:
        yield token_transform[language](data_sample[language_index[language]])

In [8]:
def yield_source_pair(src_filepath, tgt_filepath):
    raw_src_iter = io.open(src_filepath, encoding='utf8').readlines()
    raw_tgt_iter = io.open(tgt_filepath, encoding='utf8').readlines()

    for (raw_src, raw_tgt) in zip(raw_src_iter, raw_tgt_iter):
        yield (raw_src.strip(), raw_tgt.strip())

src_filepath = '/home/uih/JYL/GitHub/Transformer/data/dev_zh/newsdev2017-enzh-ref.zh.sgm.txt'
tgt_filepath = '/home/uih/JYL/GitHub/Transformer/data/dev_en/newsdev2017-zhen-ref.en.sgm.txt'
for p in yield_source_pair(src_filepath, tgt_filepath):
    print(p)
    break

('加利福尼亚州水务工程的新问题', 'New Questions Over California Water Project')


In [9]:
# define speical symbols and indices
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<bos>', '<eos>']

In [10]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    train_iter = yield_source_pair(src_filepath, tgt_filepath)
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln), min_freq=1, specials=special_symbols, special_first=True)

In [14]:
len(vocab_transform[SRC_LANGUAGE]), len(vocab_transform[TGT_LANGUAGE])

(9681, 8917)

In [25]:
for i in range(len(vocab_transform[TGT_LANGUAGE])):
    if i < 10:
        print(vocab_transform[TGT_LANGUAGE].lookup_token(i))

<unk>
<pad>
<bos>
<eos>
the
,
.
of
and
to


**Set ``UNK_IDX`` as the default index. This index is returned when the token is not found. If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.**

In [12]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

**As seen in the Data Sourcing and Processing section, our data iterator yields a pair of raw strings. 
We need to convert these string pairs into the batched tensors that can be processed by our Seq2Seq network defined previously. 
Below we define our collate function that converts a batch of raw strings into batch tensors that can be fed directly into our model.**

In [62]:
from torch.nn.utils.rnn import pad_sequence

def sequentail_transforms(*transforms):
    def func(txt_input):
        for transform in transforms:
            txt_input = transform(txt_input)
        return txt_input
    return func

def tensor_transform(token_ids: List[int]):
    return torch.cat((torch.tensor([BOS_IDX]), torch.tensor(token_ids), torch.tensor([EOS_IDX])))

text_transform = {}
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    text_transform[ln] = sequentail_transforms(token_transform[ln], vocab_transform[ln], tensor_transform)

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    print(batch)
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample))
    
    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch.permute(1, 0), tgt_batch.permute(1, 0)

**build dataloader**

In [63]:
from typing import Iterator
from torch.utils.data import IterableDataset

class IterDataset(IterableDataset):  # 可迭代对象, 要实现__iter__方法(返回一个迭代器), 不能实现__next__方法

    def __init__(self, src_filepath, tgt_filepath):
        self.src_filepath = src_filepath
        self.tgt_filepath = tgt_filepath

    def __len__(self):
        raw_src_iter = io.open(src_filepath, encoding='utf8').readlines()
        raw_tgt_iter = io.open(tgt_filepath, encoding='utf8').readlines()
        assert len(raw_src_iter) == len(raw_tgt_iter)
        return len(raw_src_iter)
    
    def __iter__(self) -> Iterator:
        return yield_source_pair(self.src_filepath, self.tgt_filepath)

In [64]:
from torch.utils.data import DataLoader

BATCH_SIZE = 10
train_iter = IterDataset(src_filepath, tgt_filepath)
train_loader = DataLoader(train_iter, batch_size=BATCH_SIZE, collate_fn=collate_fn)

In [68]:
def check_dataloader(pair, index=0):
    assert index < pair[0].shape[0]
    language_index = {SRC_LANGUAGE:0, TGT_LANGUAGE: 1}
    ss_src, ss_tgt = "", ""
    
    for i in pair[language_index[SRC_LANGUAGE]][index]:
        ss_src += vocab_transform[SRC_LANGUAGE].lookup_token(i) + " "

    for i in pair[language_index[TGT_LANGUAGE]][index]:
        ss_tgt += vocab_transform[TGT_LANGUAGE].lookup_token(i) + " "
    print(ss_src, ss_tgt, sep='\n')

pair = next(iter(train_loader))
check_dataloader(pair, 2)

[('加利福尼亚州水务工程的新问题', 'New Questions Over California Water Project'), ('在加利福尼亚州一个主要水务管理区披露州长杰瑞·布朗领导的行政当局将提供政府资金以完成两条巨型输水隧道的规划之后，有一些评论家和一位州议员表示，他们想进一步了解由谁来为州长所支持的拟耗资160亿美元的水务工程承担费用。', "Critics and a state lawmaker say they want more explanations on who's paying for a proposed $16 billion water project backed by Gov. Jerry Brown, after a leading California water district said Brown's administration was offering government funding to finish the planning for the two giant water tunnels."), ('评论家表示，洛杉矶的MWD周四所提及的政府资金可能有悖于该州长期以来的承诺，该承诺说，为了实现布朗所设想的挖掘两条35英里长隧道，以便于把水从萨克拉门托河向南输送，并主要供应给加利福尼亚州中部和南部地区的愿景，各地方水管区（而非加利福尼亚州自身）将承担费用。', "Critics said the government funding described by the Los Angeles-based Metropolitan Water District on Thursday could run counter to long-standing state assurances that various local water districts, not California itself, would pay for Brown's vision of digging twin 35-mile-long tunnels to carry water from the Sacramento River south, mainly for Central and Southern Californ